# zhsegment: default program

In [9]:
from zhsegment import *

## Run the default solution on dev

In [8]:
Pw = Pdist(data=datafile("../data/count_1w.txt"))
segmenter = Segment(Pw) # note that the default solution for this homework ignores the unigram counts
output_full = []
with open("../data/input/dev.txt") as f:
    for line in f:
        output = " ".join(segmenter.segment(line.strip()))
        output_full.append(output)
print("\n".join(output_full[:3])) # print out the first three lines of output as a sanity check

中 美 在 沪 签 订 高 科 技 合 作 协 议
新 华 社 上 海 八 月 三 十 一 日 电 （ 记 者 白 国 良 、 夏 儒 阁 ）
“ 中 美 合 作 高 科 技 项 目 签 字 仪 式 ” 今 天 在 上 海 举 行 。


## Evaluate the default output

In [9]:
from zhsegment_check import fscore
with open('../data/reference/dev.out', 'r') as refh:
    ref_data = [str(x).strip() for x in refh.read().splitlines()]
    tally = fscore(ref_data, output_full)
    print("score: {:.2f}".format(tally), file=sys.stderr)

score: 0.27


In [10]:
class Segment:

    def __init__(self, Pw):
        self.Pw = Pw

    
    def segment(self, text):
        "Return a list of words that is the best segmentation of text."
        if not text: return []
        candidates = ([first]+self.segment(rem) for first,rem in self.splits(text))
        return max(candidates, key=self.Pwords)

    def splits(self, text, L=20):
        "Return a list of all possible (first, rem) pairs, len(first)<=L."
        return [(text[:i+1], text[i+1:]) 
                for i in range(min(len(text), L))]

    def Pwords(self, words): 
        "The Naive Bayes probability of a sequence of words."
        return product(self.Pw(w) for w in words)


In [11]:
Pw = Pdist(data=datafile("../data/count_1w.txt"))
segmenter = Segment(Pw) # note that the default solution for this homework ignores the unigram counts
output_full = []
output_orig = []
with open("../data/input/dev.txt") as f:
    for line in f:
        output_orig.append(line)
        output = min(segmenter.splits(line), key=segmenter.Pwords)
        output_full.append(output)
print(output_orig[0])
print(output_full[0]) # print out the first three lines of output as a sanity check

中美在沪签订高科技合作协议

('中美', '在沪签订高科技合作协议\n')


In [38]:
segmenter.Pwords(output_orig[0])

2.955908783605565e-57

## Documentation

Write some beautiful documentation of your program here.

## Implement class Entry

#### For the convenience to insert ENTRIES into the heap, we create a structure name "Entry", including word, starting point, log-probability of the word, backpointer for getting the best segmentation recursively

In this structure, we override the functions for comparing two Entries, but this is a little different from the normal ones. 

For the 'equal' operation, we also implement a condition that two compared objects are not the same type, since the entry including the first word does not have back-pointer and we need to compare entry with None to figure out whether the program has achieved the first word.

For the 'not equal' operation, we reverse the comparison operators, since the heap provided by Python is min-heap but we want the entry with higher probability to be at the top of the heap.

In [4]:
class Entry:
    def __init__(self, word, startpt, logP, backpointer):
        self.word = word
        self.startpt = startpt
        self.logP = logP
        self.backpointer = backpointer

    def __eq__(self, entry):
        if not isinstance(entry, type(self)): return False
        return (self.word == entry.word) and (self.startpt == entry.startpt) and (self.logP == entry.logP)

    def __gt__(self, entry):
        return self.logP <= entry.logP

    def __lt__(self, entry):
        return self.logP > entry.logP

## Implemented pseudocode

In addition to the version indicated on the website, we add some other constraints to get a higher accuracy and accelerate the program.

Due to the specific features of Chinese words, it is rare to have more than 4 single words in one segmentation especially with such a content in the *dev.txt*. The *dev.txt* includes contents related to news and it is hard to have a word with more than 4 letters. So we contraint the number of letters in the missing word. We only take the missing words with less than and equal 4 letters into account. This accelerates the program dramatically, from manually interruption to less than 1 second.

#### Version 1 (using only unigram)

This version only use unigram to implement the segmenter. 

The accuracy for this implementation is 0.92 (this accuracy includes the modification of the probability of missing word, which will be discussed after the implementation of the pseudocode).

In [ ]:
class Segment:

    def __init__(self, Pw):
        self.Pw = Pw

    def segment(self, text):
        ### Initialize the heap
        heap = []
        for i in range(len(text)):
            word = text[0:i+1]
            if word in self.Pw or len(word) <= 4:
                heapq.heappush(heap, Entry(word, 0, log10(self.Pw(word)), None))
                
        ### Iteratively fill in chart[i] for all i
        chart = {}
        for i in range(len(text)):
            chart[i] = Entry(None, None, None, None)
        
        while len(heap) > 0:
            entry = heapq.heappop(heap)
            endindex = entry.startpt + len(entry.word) - 1
            if chart[endindex].backpointer is not None:
                preventry = chart[endindex].backpointer
                if entry.logP > preventry.logP:
                    chart[endindex] = entry
                if entry.logP <= preventry.logP:
                    continue
            else:
                chart[endindex] = entry
                
            for i in range(endindex + 1, len(text)):
                word = text[endindex + 1 : i + 1]
                if word in self.Pw or len(word) <= 4:
                    newentry = Entry(word, endindex + 1, entry.logP + log10(self.Pw(word)), entry)
                    if newentry not in heap:
                        heapq.heappush(heap, newentry)
            
        ### Get the best segmentation
        finalindex = len(chart)
        finalentry = chart[finalindex - 1]
        segmentation = []
        while finalentry != None:
            segmentation.insert(0, finalentry.word)
            finalentry = finalentry.backpointer

        return segmentation



#### Version 2 (combined with bigram)

We combine unigram model and bigram model to score word segmentation candidates. We only use bigram model for those words that both words are in the vocabulary since we can continue to use the penalty for long words that we used in the first version and we do not need to do smoothing for bigram model.

We expected a higher accuracy from this model. Unfortunately, we can not make a progress on the accuracy. The accuracy of this model is still 0.92. We will analysis the possbile reason for this situation in the Analysis part.

In [ ]:
class Segment:

    def __init__(self, uniPw, biPw):
        self.uniPw = uniPw
        self.biPw = biPw

    def segment(self, text):
        ### Initialize the heap
        heap = []
        for i in range(len(text)):
            word = text[0:i+1]
            if word in self.uniPw or len(word) <= 4:
                heapq.heappush(heap, Entry(word, 0, log10(self.uniPw(word)), None))

        ### Iteratively fill in chart[i] for all i
        chart = {}
        for i in range(len(text)):
            chart[i] = Entry(None, None, None, None)

        while len(heap) > 0:
            entry = heapq.heappop(heap)
            endindex = entry.startpt + len(entry.word) - 1
            if chart[endindex].backpointer is not None:
                preventry = chart[endindex].backpointer
                if entry.logP > preventry.logP:
                    chart[endindex] = entry
                if entry.logP <= preventry.logP:
                    continue
            else:
                chart[endindex] = entry

            for i in range(endindex + 1, len(text)):
                word = text[endindex + 1 : i + 1]
                if word in self.uniPw or len(word) <= 4:
                    wordPair = entry.word + " " + word
                    uniP = self.uniPw(word)

                    if wordPair in self.biPw and entry.word in self.uniPw:
                        biP = self.biPw(wordPair) / self.uniPw(word)
                        newentry = Entry(word, endindex + 1, entry.logP + log10(biP), entry)
                    else:
                        newentry = Entry(word, endindex + 1, entry.logP + log10(uniP), entry)
                    if newentry not in heap:
                        heapq.heappush(heap, newentry)

        ### Get the best segmentation
        finalindex = len(chart)
        finalentry = chart[finalindex - 1]
        segmentation = []
        while finalentry != None:
            segmentation.insert(0, finalentry.word)
        finalentry = finalentry.backpointer

        return segmentation


## Modify probability for missing words

Because of the features of Chinese words, the probability of long words decreases dramatically as the number of words increases. As a result, we use 

```1./(Total number of words * 9000 ** the length of the word*```

as the probability of the missing words.

In [ ]:
class Pdist(dict):
    "A probability distribution estimated from counts in datafile."
    def __init__(self, data=[], N=None, missingfn=None):
        for key,count in data:
            self[key] = self.get(key, 0) + int(count)
        self.N = float(N or sum(self.values()))
        self.missingfn = missingfn or (lambda k, N: 1./(N * 9000 ** len(k)))
    def __call__(self, key): 
        if key in self: return self[key]/self.N  
        else: return self.missingfn(key, self.N)

## Run check.py

In [10]:
uniPw = Pdist(data=datafile("../data/count_1w.txt"))
biPw = Pdist(data=datafile("../data/count_1w.txt")) 
segmenter = Segment(uniPw, biPw)
output_full = []
with open("../data/input/dev.txt") as f:
    for line in f:
        output = " ".join(segmenter.segment(line.strip()))
        output_full.append(output)
print("\n".join(output_full[:3])) # print out the first three lines of output as a sanity check

中 美 在 沪 签订 高 科技 合作 协议
新华社 上海 八月 三十一日 电 （ 记者 白 国 良 、 夏儒阁 ）
“ 中 美 合作 高 科技 项目 签字 仪式 ” 今天 在 上海 举行 。


In [11]:
from zhsegment_check import fscore
with open('../data/reference/dev.out', 'r') as refh:
    ref_data = [str(x).strip() for x in refh.read().splitlines()]
    tally = fscore(ref_data, output_full)
    print("score: {:.2f}".format(tally), file=sys.stderr)

score: 0.92


## Analysis

Do some analysis of the results. What ideas did you try? What worked and what did not?

## Analysis of the reason why the improvement of model can not improve the accuracy

Due to our implementation of the model, we want to use bigram model for those words that are already in the vocabulary. We expected some improvement of the accuracy, but the result dispointed us. 

But after checking the *output/dev.out* and *reference/dev.out*, we found that the dispointing result is reasonable and comprehensible. We found that most words that have been predicted wrong are names. 

For exmaple, *白国良* has been segmented into *白 国 良* and *朱迪·梅罗* has been segmented into *朱 迪·梅罗*.

For the both cases, bigram model can not help improve the accuracy since those names do not appear neither in the vocabulary file for the unigram nor in the vocabulary file for the bigram. And the reason why *朱* has been seperated is that it appears in the vocabulary list for the unigram.